In [82]:
import numpy as np
import pandas as pd
import os
import time
import gc
import random

# from nltk.chunk.named_entity import build_model
from tqdm.notebook import tqdm_notebook as tqdm
from tensorflow.keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F


torch.backends.cudnn.enabled = False

In [2]:
NUM_MODELS = 1
N_EPOCHS = 1
BATCH_SIZE = 32
# units parameters in Keras.layers.LSTM/cuDNNLSTM
# the dimension of the output vector of each LSTM cell.
LSTM_UNITS = 2 #128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 8 #220

In [3]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    #each line in the file looks like
    # apple 0.3 0.4 0.5 0.6 ...
    # that is a word followed by 50 float numbers
    with open(path) as f:
        # import pdb; pdb.set_trace()
        return dict(get_coefs(*line.strip().split(' ')) for line in f) #used in python
        # return dict(get_coefs(*o.strip().split(" ")) for o in tqdm(f))

def build_matrix(word_index, path):
    #path: a path that contains embedding matrix
    #word_index is a dict of the form ('apple': 123, 'banana': 349, etc)
    # that means word_index[word] gives the index of the word
    # word_index was built from all commment_texts
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 50))
    unknown_words = []

    # word_index is a dict. Each element is (word:i) where i is the index
    # of the word
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)

    return embedding_matrix, unknown_words

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
def train_model(model, train, test, loss_fn, output_dim, lr=0.001,
                batch_size=BATCH_SIZE, n_epochs=N_EPOCHS,
                enable_checkpoint_ensemble=True):
    param_lrs = [{'params': param, 'lr': lr} for param in model.parameters()]
    optimizer = torch.optim.Adam(param_lrs, lr=lr)

    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.6 ** epoch)

    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    all_test_preds = []
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]

    print("First Epoch")

    for epoch in range(n_epochs):
        start_time = time.time()

        model.train()
        avg_loss = 0.

        for data in tqdm(train_loader, disable=False):
            x_batch = data[:-1]
            y_batch = data[-1]

            y_pred = model(*x_batch)

            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item() / len(train_loader)

        scheduler.step()

        model.eval()
        test_preds = np.zeros((len(test), output_dim))

        for i, x_batch in enumerate(test_loader):
            y_pred = sigmoid(model(*x_batch).detach().cpu().numpy())

            test_preds[i * batch_size:(i + 1) * batch_size, :] = y_pred

        all_test_preds.append(test_preds)
        elapsed_time = time.time() - start_time
        print('Epoch {}/{} \t loss={:.4f} \t time={:.2f}s'.format(
            epoch + 1, n_epochs, avg_loss, elapsed_time))

    if enable_checkpoint_ensemble:
        test_preds = np.average(all_test_preds, weights=checkpoint_weights, axis=0)
    else:
        test_preds = all_test_preds[-1]

    return test_preds


In [5]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)  # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x


In [6]:
class NeuralNet(nn.Module):
    def __init__(self, embedding_matrix, num_aux_targets):
        super(NeuralNet, self).__init__()
        
        embed_size = embedding_matrix.shape[1] # 50

        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)

        self.lstm1 = nn.LSTM(embed_size, LSTM_UNITS, bidirectional=True, batch_first=True)
        self.lstm2 = nn.LSTM(LSTM_UNITS * 2, LSTM_UNITS, bidirectional=True, batch_first=True)

        self.linear1 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)
        self.linear2 = nn.Linear(DENSE_HIDDEN_UNITS, DENSE_HIDDEN_UNITS)

        self.linear_out = nn.Linear(DENSE_HIDDEN_UNITS, 1)
        self.linear_aux_out = nn.Linear(DENSE_HIDDEN_UNITS, num_aux_targets)

    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = self.embedding_dropout(h_embedding)

        h_lstm1, _ = self.lstm1(h_embedding)
        h_lstm2, _ = self.lstm2(h_lstm1)

        # global average pooling
        avg_pool = torch.mean(h_lstm2, 1)
        # global max pooling
        max_pool, _ = torch.max(h_lstm2, 1)

        h_conc = torch.cat((max_pool, avg_pool), 1)
        h_conc_linear1 = F.relu(self.linear1(h_conc))
        h_conc_linear2 = F.relu(self.linear2(h_conc))

        hidden = h_conc + h_conc_linear1 + h_conc_linear2

        result = self.linear_out(hidden)
        aux_result = self.linear_aux_out(hidden)
        out = torch.cat([result, aux_result], 1)

        return out

In [7]:
def preprocess(data):
    '''
    Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
    '''
    punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
    def clean_special_chars(text, punct):
        for p in punct:
            text = text.replace(p, ' ')
        return text


    d = pd.Series(data.astype(str))
    data = d.apply(lambda x: clean_special_chars(x, punct))

    return data

In [49]:
train = pd.read_csv('jigsaw-unintended-bias-in-toxicity-classification/train.csv')[:1000]
test = pd.read_csv('jigsaw-unintended-bias-in-toxicity-classification/test.csv')

x_train = preprocess(train['comment_text'].fillna('').values)
y_train = np.where(train['target'] >= 0.5, 1, 0)

y_aux_train = train[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
x_test = preprocess(test['comment_text'].fillna('').values)

In [50]:
print(train.shape)
print(test.shape)
print(x_train.shape)
print(y_train.shape)
print(y_aux_train.shape)
print(x_test.shape)

(1000, 45)
(97320, 2)
(1000,)
(1000,)
(1000, 6)
(97320,)


In [51]:
max_features = None

# tokenizer is a class with some method
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(list(x_train) + list(x_test))

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

max_features = max_features or len(tokenizer.word_index) + 1

In [52]:
print(len(x_train))
print(len(x_test))
print(x_train[0])
print(x_test[0])

1000
97320
[21, 6, 35, 2056, 9, 13, 53, 45, 11, 107, 39, 1022, 2, 203, 21, 126, 163, 347, 97, 220]
[1823, 344, 8, 11, 132, 39, 5491, 110, 21, 1288, 2, 154, 54, 119]


In [53]:
#https://keras.io/preprocessing/sequence/
# https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/sequence.py
#each comment_text is now a list of word

x_train = sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)

In [54]:
print(x_train.shape)
print(x_test.shape)

(1000, 8)
(97320, 8)


In [55]:
MODEL_NAME = "inter_poly_debiased"

EMBEDDING_FILES = 'DebiasMulticlassWordEmbedding-master/Debiasing/output/intersection_evalset/poly/reddit_US_txt_tok_clean_cleanedforw2v_0_inter_hardDebiasedEmbeddingsOut.w2v'#'./input/w2vs/intersection_debias/poly/reddit_US_txt_tok_clean_cleanedforw2v_0_inter_hardDebiasedEmbeddingsOut.w2v'

In [56]:
embedding_matrix, unknown_words = build_matrix(tokenizer.word_index, EMBEDDING_FILES) # (284526, 50)

In [57]:
print('n unknown words: ', len(unknown_words))
print(embedding_matrix.shape)

n unknown words:  39426
(74920, 50)


In [58]:
y_aux_train.shape[-1]

6

In [59]:
model = NeuralNet(embedding_matrix, y_aux_train.shape[-1]) # (327009, 50), 6
# model.cuda()

In [65]:
# y_train_concat = torch.cat((y_train.unsqueeze(1), y_aux_train), dim=1)
y_train_concat = np.concatenate((y_train[:, np.newaxis], y_aux_train), axis=1)
print(y_train_concat.shape)
print(y_train_concat[1])

(1000, 7)
[0. 0. 0. 0. 0. 0. 0.]


In [66]:
from torch.utils.data import TensorDataset
# convert data to PyTorch tensors
x_train1 = torch.tensor(x_train, dtype=torch.int64)
y_train1 = torch.tensor(y_train_concat, dtype=torch.float)
x_test1 = torch.tensor(x_test, dtype=torch.int64)

# create PyTorch datasets
train_dataset = TensorDataset(x_train1, y_train1)
test_dataset = TensorDataset(x_test1)

print(train_dataset[1])
# print(test_dataset)

(tensor([ 57,  18, 149, 195,  61,   7,  39,  96]), tensor([0., 0., 0., 0., 0., 0., 0.]))


In [73]:
# test_preds = train_model(model, train_dataset, test_dataset, output_dim=y_train_torch.shape[-1],
#                             loss_fn=nn.BCEWithLogitsLoss(reduction='mean'), n_epochs=1)


test_preds = train_model(model, train_dataset, test_dataset, output_dim=y_train_concat.shape[-1],
                            loss_fn=nn.BCEWithLogitsLoss(reduction='mean'), n_epochs=1)


First Epoch


  0%|          | 0/32 [00:00<?, ?it/s]

Epoch 1/1 	 loss=0.4324 	 time=14.73s


In [74]:
torch.save(model.state_dict(), "./{}_1.h5".format(MODEL_NAME))

In [ ]:
# for model_idx in range(NUM_MODELS):
#     print('Model ', model_idx)
#     # seed_everything(1234 + model_idx)
#     model = NeuralNet(embedding_matrix, y_aux_train.shape[-1]) # (327009, 50), 6
#     model.cuda()

#     test_preds = train_model(model, train_dataset, test_dataset, output_dim=y_train_torch.shape[-1],
#                              loss_fn=nn.BCEWithLogitsLoss(reduction='mean'), n_epochs=1)
#     torch.save(model.state_dict(), "./{}_{}.h5".format(MODEL_NAME, model_idx))
    
#     all_test_preds.append(test_preds)


In [75]:
selected_index = 0
test_model = NeuralNet(embedding_matrix, y_aux_train.shape[-1])
test_model.load_state_dict(torch.load("inter_poly_debiased_1.h5".format(MODEL_NAME, selected_index)))
# test_model = test_model.cuda()
test_model.eval()

NeuralNet(
  (embedding): Embedding(74920, 50)
  (embedding_dropout): SpatialDropout(p=0.3, inplace=False)
  (lstm1): LSTM(50, 2, batch_first=True, bidirectional=True)
  (lstm2): LSTM(4, 2, batch_first=True, bidirectional=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
  (linear_aux_out): Linear(in_features=8, out_features=6, bias=True)
)

In [76]:
# del x_train

df_test1 = pd.read_csv('jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv')
df_test2 = pd.read_csv('jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv')
frames = [df_test1, df_test2]
df_test = pd.concat(frames)
df_test["target"] = df_test["toxicity"].apply(lambda x: 1 if x >= 0.5 else 0)
df_test["y_pred"] = ""

In [77]:
print(df_test1.shape)
print(df_test2.shape)
print(df_test.shape)

(97320, 45)
(97320, 45)
(194640, 47)


In [78]:
x_test = preprocess(df_test['comment_text'].fillna('').values)
y_test = np.where(df_test['target'] >= 0.5, 1, 0)
print(x_test.shape)
print(y_test.shape)

(194640,)
(194640,)


In [79]:
max_features = None
max_features = max_features or len(tokenizer.word_index) + 1

x_test = tokenizer.texts_to_sequences(x_test)
print(len(x_test))


194640


In [80]:
#https://keras.io/preprocessing/sequence/
# https://github.com/keras-team/keras-preprocessing/blob/master/keras_preprocessing/sequence.py
#each comment_text is now a list of word

x_test = sequence.pad_sequences(x_test, maxlen=MAX_LEN)
# x_test_torch = torch.tensor(x_test, dtype=torch.long).cuda()
x_test_torch = torch.tensor(x_test, dtype=torch.long)

print(x_test.shape)
print(x_test_torch.shape)

(194640, 8)
torch.Size([194640, 8])


In [88]:
# Training
batch_size=32
test_dataset = data.TensorDataset(x_test_torch)
testset = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


print(len(test_dataset))
print(test_dataset[:2])
print(testset)

194640
(tensor([[5491,  110,   21, 1288,    2,  154,   54,  119],
        [   3,    1,  744,   20,   14, 2640,   59,  100]]),)


In [86]:
test_preds = np.zeros((len(test_dataset), np.hstack([y_train[:, np.newaxis], y_aux_train]).shape[-1]))
print(len(test_preds))
print(test_preds[:2])

194640
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [89]:
import gc
gc.collect()
# torch.cuda.empty_cache()

for i, x_batch in enumerate(testset):
    y_pred = sigmoid(test_model(*x_batch).detach().cpu().numpy())
    if x_batch[0].shape[0] == 32:
        test_preds[i * batch_size:(i + 1) * batch_size, :] = y_pred
    else:
        test_preds[i * batch_size:, :] = y_pred

df_test.loc[:, "y_pred"] = list(test_preds[:, 0])
df_test.y_pred = df_test.y_pred.apply(lambda x: 1 if x>=0.5 else 0)
print(df_test.shape)
print(df_test.columns)

(194640, 47)
Index(['id', 'comment_text', 'created_date', 'publication_id', 'parent_id',
       'article_id', 'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
       'toxicity', 'severe_toxicity', 'obscene', 'sexual_explicit',
       'identity_attack', 'insult', 'threat', 'identity_annotator_count',
       'toxicity_annotator_count', 'male', 'female', 'transgender',
       'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual',
       'other_sexual_orientation', 'christian', 'jewish', 'muslim', 'hindu',
       'buddhist', 'atheist', 'other_religion', 'black', 'white', 'asian',
       'latino', 'other_race_or_ethnicity', 'physical_disability',
       'intellectual_or_learning_disability', 'psychiatric_or_mental_illness',
       'other_disability', 'target', 'y_pred'],
      dtype='object')


C:\Users\Hammad\AppData\Local\Temp\ipykernel_5676\683290888.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_test.loc[:, "y_pred"] = list(test_preds[:, 0])


In [90]:
fnr = len(df_test[(df_test.target == 1)&(df_test.y_pred == 0)])/len(df_test)
fpr = len(df_test[(df_test.target == 0)&(df_test.y_pred == 1)])/len(df_test)

group1 = [ 'male', 'female', 'transgender', 'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']
df_group1 = []
for column in group1:
    df = df_test[df_test[column] == 1]
    if len(df) == 0:
        df = df_test[np.isnan(df_test[column])]
    df_group1.append(df)

group2 = [ 'christian', 'jewish', 'muslim', 'hindu', 'buddhist', 'atheist', 'other_religion']
df_group2 = []
for column in group2:
    df = df_test[df_test[column] == 1]
    if len(df) == 0:
        df = df_test[np.isnan(df_test[column])]
    df_group2.append(df)

group3 = [ 'black', 'white', 'asian', 'latino', 'other_race_or_ethnicity']
df_group3 = []
for column in group3:
    df = df_test[df_test[column] == 1]
    if len(df) == 0:
        df = df_test[np.isnan(df_test[column])]
    df_group3.append(df)

fnr_group1 = []
fpr_group1 = []
for i, df in enumerate(df_group1):
    if len(df) == 0:
        print(i)
    g_fnr = len(df[(df.target == 1)&(df.y_pred == 0)])/len(df)
    g_fpr = len(df[(df.target == 0)&(df.y_pred == 1)])/len(df)
    fnr_group1.append(g_fnr)
    fpr_group1.append(g_fpr)

fnr_group2 = []
fpr_group2 = []
for i, df in enumerate(df_group2):
    if len(df) == 0:
        print(i)
    g_fnr = len(df[(df.target == 1)&(df.y_pred == 0)])/len(df)
    g_fpr = len(df[(df.target == 0)&(df.y_pred == 1)])/len(df)
    fnr_group2.append(g_fnr)
    fpr_group2.append(g_fpr)

fnr_group3 = []
fpr_group3 = []
for i, df in enumerate(df_group3):
    if len(df) == 0:
        print(i)
    g_fnr = len(df[(df.target == 1)&(df.y_pred == 0)])/len(df)
    g_fpr = len(df[(df.target == 0)&(df.y_pred == 1)])/len(df)
    fnr_group3.append(g_fnr)
    fpr_group3.append(g_fpr)

fned_group1 = 0
fped_group1 = 0
for f in fnr_group1:
    fned_group1 += abs(fnr-f)
for f in fpr_group1:
    fped_group1 += abs(fpr-f)

fned_group2 = 0
fped_group2 = 0
for f in fnr_group2:
    fned_group2 += abs(fnr-f)
for f in fpr_group2:
    fped_group2 += abs(fpr-f)

fned_group3 = 0
fped_group3 = 0
for f in fnr_group3:
    fned_group3 += abs(fnr-f)
for f in fpr_group3:
    fped_group3 += abs(fpr-f)

print("{}".format(MODEL_NAME))
print("FPED group 1 gender : {:.4f}".format(fped_group1))
print("FNED group 1 gender : {:.4f}".format(fned_group1))
print("FPED group 2 religion : {:.4f}".format(fped_group2))
print("FNED group 2 religion : {:.4f}".format(fned_group2))
print("FPED group 3 race : {:.4f}".format(fped_group3))
print("FNED group 3 race : {:.4f}".format(fned_group3))

inter_poly_debiased
FPED group 1 gender : 0.0000
FNED group 1 gender : 1.1713
FPED group 2 religion : 0.0000
FNED group 2 religion : 0.3619
FPED group 3 race : 0.0000
FNED group 3 race : 0.8028
